In [3]:
import pandas as pd
import numpy as np  
import sys
sys.path.append('../src')

In [4]:
# Load the cleaned data
from data_processing import load_csv
cleaned_data= load_csv('../data/processed/cleaned_data.csv')
output_data = load_csv('../data/processed/output.csv')


In [25]:
output_data.head()

,CustomerId,total_amount,customer_transactions,fraud_transactions,total_fraud_amount,fraud_ratio,fraud_amount_ratio,fraud_frequency_score,high_value_fraud,last_transaction_date
0,CustomerId_4406,10000.0,119,NaN,NaN,NaN,NaN,NaN,0,2019-02-12 10:24:40+00:00
1,CustomerId_4683,10000.0,2,NaN,NaN,NaN,NaN,NaN,0,2018-11-24 08:30:25+00:00
2,CustomerId_988,30400.0,38,NaN,NaN,NaN,NaN,NaN,0,2019-02-08 07:53:33+00:00
3,CustomerId_1432,4775.0,1,NaN,NaN,NaN,NaN,NaN,0,2018-11-15 03:35:10+00:00
4,CustomerId_2858,32000.0,29,NaN,NaN,NaN,NaN,NaN,0,2019-02-08 06:51:39+00:00


In [38]:
cleaned_data['TransactionStartTime'] = pd.to_datetime(cleaned_data['TransactionStartTime'])
#get last transaction date in cleaned_data
ref_date = cleaned_data['TransactionStartTime'].max() + pd.Timedelta(days=1)


In [39]:
output_data.columns

Index(['CustomerId', 'total_amount', 'customer_transactions',
       'fraud_transactions', 'total_fraud_amount', 'fraud_ratio',
       'fraud_amount_ratio', 'fraud_frequency_score', 'high_value_fraud',
       'last_transaction_date'],
      dtype='object')

In [40]:
rfm = cleaned_data.groupby('CustomerId').agg({
    'TransactionStartTime': lambda x: (ref_date - x.max()).days,  # Recency
    'TransactionId': 'count',                                     # Frequency
    'Amount': 'sum'                                               # Monetary
}).reset_index()


In [41]:
output_data['last_transaction_date'] = pd.to_datetime(output_data['last_transaction_date'])

ref_date = output_data['last_transaction_date'].max() + pd.Timedelta(days=1)
output_data['Recency'] = (ref_date - output_data['last_transaction_date']).dt.days
output_data['Frequency'] = output_data['customer_transactions']
output_data['Monetary'] = output_data['total_amount']


In [42]:
# print only rmf cols  head of output_data
rmf = output_data[['CustomerId', 'Recency', 'Frequency', 'Monetary']]
print(rmf.head())


        CustomerId  Recency  Frequency  Monetary
0  CustomerId_4406        1        119   10000.0
1  CustomerId_4683       82          2   10000.0
2   CustomerId_988        6         38   30400.0
3  CustomerId_1432       91          1    4775.0
4  CustomerId_2858        6         29   32000.0


In [43]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

rfm_scaled = StandardScaler().fit_transform(output_data[['Recency', 'Frequency', 'Monetary']])
kmeans = KMeans(n_clusters=3, random_state=42)
output_data['cluster'] = kmeans.fit_predict(rfm_scaled)


In [44]:
cluster_profile = output_data.groupby('cluster')[['Recency', 'Frequency', 'Monetary']].mean()
high_risk_cluster = cluster_profile.sort_values(['Recency', 'Frequency', 'Monetary'], ascending=[False, True, True]).index[0]
output_data['credit_risk'] = output_data['cluster'].apply(lambda x: 1 if x == high_risk_cluster else 0)


In [46]:
output_data.head()

,CustomerId,total_amount,customer_transactions,fraud_transactions,total_fraud_amount,fraud_ratio,fraud_amount_ratio,fraud_frequency_score,high_value_fraud,last_transaction_date,Recency,Frequency,Monetary,cluster,credit_risk
0,CustomerId_4406,10000.0,119,NaN,NaN,NaN,NaN,NaN,0,2019-02-12 10:24:40+00:00,1,119,10000.0,1,0
1,CustomerId_4683,10000.0,2,NaN,NaN,NaN,NaN,NaN,0,2018-11-24 08:30:25+00:00,82,2,10000.0,0,1
2,CustomerId_988,30400.0,38,NaN,NaN,NaN,NaN,NaN,0,2019-02-08 07:53:33+00:00,6,38,30400.0,1,0
3,CustomerId_1432,4775.0,1,NaN,NaN,NaN,NaN,NaN,0,2018-11-15 03:35:10+00:00,91,1,4775.0,0,1
4,CustomerId_2858,32000.0,29,NaN,NaN,NaN,NaN,NaN,0,2019-02-08 06:51:39+00:00,6,29,32000.0,1,0


In [35]:
# save output_df to csv
output_data.to_csv('../data/processed/output2.csv', index=False)